<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [133]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [134]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [135]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[2]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[2] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [136]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [137]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [138]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,0.999601,0.999309,-0.299280,11.708970,0.222346,9.967150,0.085353,0.469594,-49.995722
1,-0.894733,0.995152,-0.190358,-1.851966,-0.315801,18.121831,0.456991,0.557295,179.443563
2,0.290020,0.999978,0.086489,67.740657,0.161528,1.746497,0.002306,0.401032,2.716082
3,-0.486449,0.996483,-0.992567,2.999677,0.575906,4.522174,-0.113105,0.737967,2.464585
4,0.799599,0.999988,-0.017279,6.960273,-0.556539,0.695879,0.102726,0.726808,6.726555
...,...,...,...,...,...,...,...,...,...
95,-0.188338,1.000000,0.580439,10.323145,0.551018,0.991546,-0.007918,0.723571,-0.274449
96,-0.706395,0.989278,-0.597419,-0.607748,0.457498,4.189751,0.976895,0.664609,5.949951
97,0.913580,0.999885,-0.998308,0.995927,0.306463,-0.273975,0.876784,0.549261,0.278633
98,0.755703,0.998418,-0.958678,-1.384515,-0.111247,0.944640,-0.474493,0.333194,-0.831356


In [139]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,0.628991,0.999940,0.791910,5.251056,-0.104969,2.337276,0.094999,0.323691,1.648289
1,-0.635573,0.999504,-0.804946,-2.294004,0.087674,5.680739,0.220878,0.295908,5.121700
2,0.921568,0.999991,-0.309029,-2.811435,-0.484635,9.275625,-0.314676,0.682558,-40.894470
3,-0.772889,0.981813,0.961093,4.445289,0.178597,-0.685796,-0.152853,0.421484,0.499526
4,0.655696,0.985179,-0.847572,91.318382,-0.112231,0.572686,0.005814,0.334657,-0.502304
...,...,...,...,...,...,...,...,...,...
95,0.170849,0.986038,0.166259,-2.591348,-0.165422,0.742192,-0.027252,0.405794,0.476238
96,0.648019,0.995419,-0.058905,-1.278199,0.590957,2.227772,-0.408115,0.746429,35.129017
97,-0.004467,0.999586,-0.891856,106.052750,-0.265214,4.459878,-0.000003,0.511972,0.025161
98,0.417349,0.999999,0.634345,532.678731,-0.017714,4.013790,0.000506,0.133092,2.281149


##1. Линейная форма функции пересчёта

In [140]:
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  coef = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  estimate_linear = []
  variance_linear = []
  error_linear = []

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [153]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1  -0.026729
c11  0.183620
b11 -0.000010


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,0.999601,11.708970,0.628991,0.156706,0.002231,0.472441
1,1,-0.894733,-1.851966,-0.635573,-0.191003,0.001976,0.444717
2,1,0.290020,67.740657,0.921568,0.025878,0.008023,0.895986
3,1,-0.486449,2.999677,-0.772889,-0.116080,0.004314,0.657026
4,1,0.799599,6.960273,0.655696,0.120027,0.002869,0.535846
...,...,...,...,...,...,...,...
95,1,-0.188338,10.323145,0.170849,-0.061410,0.000539,0.232336
96,1,-0.706395,-0.607748,0.648019,-0.156432,0.006471,0.804716
97,1,0.913580,0.995927,-0.004467,0.141013,0.000212,0.145529
98,1,0.755703,-1.384515,0.417349,0.112046,0.000932,0.305403


In [154]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.4454507220487671
A = 58.07627796666762


###Для функции $x_2[t+1]$

In [143]:
#x2, x3, v1
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['x2(9)', 'x3(9)', 'v1(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))
X2_linearTable

               X2
a2   1.101665e+00
c22 -1.050328e-01
c23  4.512024e-04
b21 -5.428668e-07


,optional_col,x2(9),x3(9),v1(9),x2(10),regression,variance,"error, %"
0,1,0.999309,-0.299280,11.708970,0.999940,0.996563,1.139955e-07,0.003376
1,1,0.995152,-0.190358,-1.851966,0.999504,0.997056,5.991207e-08,0.002448
2,1,0.999978,0.086489,67.740657,0.999991,0.996637,1.125278e-07,0.003355
3,1,0.996483,-0.992567,2.999677,0.981813,0.996552,2.172358e-06,0.014739
4,1,0.999988,-0.017279,6.960273,0.985179,0.996622,1.309282e-06,0.011442
...,...,...,...,...,...,...,...,...
95,1,1.000000,0.580439,10.323145,0.986038,0.996888,1.177322e-06,0.010850
96,1,0.989278,-0.597419,-0.607748,0.995419,0.997489,4.285506e-08,0.002070
97,1,0.999885,-0.998308,0.995927,0.999586,0.996193,1.150981e-07,0.003393
98,1,0.998418,-0.958678,-1.384515,0.999999,0.996366,1.319573e-07,0.003633


In [144]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 3.263561761263342e-05
A = 0.40915767463109476


###Для функции $x_3[t+1]$

In [145]:
#x3, v1, v3
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))
X3_linearTable

           X3
a3  -0.187917
c33 -0.025868
b31 -0.000061
b33  0.009774


,optional_col,x3(9),v1(9),v3(9),x3(10),regression,variance,"error, %"
0,1,-0.299280,11.708970,9.967150,0.791910,-0.083469,0.007663,0.875465
1,1,-0.190358,-1.851966,18.121831,-0.804946,-0.005761,0.006387,0.799264
2,1,0.086489,67.740657,1.746497,-0.309029,-0.177198,0.000174,0.131843
3,1,-0.992567,2.999677,4.522174,0.961093,-0.118224,0.011649,1.079423
4,1,-0.017279,6.960273,0.695879,-0.847572,-0.181091,0.004442,0.666546
...,...,...,...,...,...,...,...,...
95,1,0.580439,10.323145,0.991546,0.166259,-0.193867,0.001297,0.360162
96,1,-0.597419,-0.607748,4.189751,-0.058905,-0.131476,0.000053,0.072578
97,1,-0.998308,0.995927,-0.273975,-0.891856,-0.164830,0.005286,0.727097
98,1,-0.958678,-1.384515,0.944640,0.634345,-0.153800,0.006212,0.788223


In [146]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.4404789895374945
A = 57.82327536943369


###Для функции $y_1[t]$

In [147]:
#x1, x2, v1
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))
Y1_linearTable

            Y1
a1  -34.470829
c11   0.898975
c12  34.055201
b11   0.000177


,optional_col,x1(9),x2(9),v1(9),y1(9),regression,variance,"error, %"
0,1,0.999601,0.999309,11.708970,0.085353,0.461521,0.001415,0.143005
1,1,-0.894733,0.995152,-1.851966,0.456991,-1.385398,0.033944,0.700407
2,1,0.290020,0.999978,67.740657,0.002306,-0.143689,0.000213,0.055502
3,1,-0.486449,0.996483,2.999677,-0.113105,-0.972160,0.007380,0.326580
4,1,0.799599,0.999988,6.960273,0.102726,0.304012,0.000405,0.076521
...,...,...,...,...,...,...,...,...
95,1,-0.188338,1.000000,10.323145,-0.007918,-0.583117,0.003309,0.218669
96,1,-0.706395,0.989278,-0.607748,0.976895,-1.415900,0.057255,0.909651
97,1,0.913580,0.999885,0.995927,0.876784,0.401910,0.002255,0.180529
98,1,0.755703,0.998418,-1.384515,-0.474493,0.209600,0.004680,0.260066


In [148]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 10.314134858582982
A = 46.82601814941087


###Для функции $y_2[t]$

In [149]:
#v3
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v3(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))
Y2_linearTable

           Y2
a2   0.505159
b23  0.000988


,optional_col,v3(9),y2(9),regression,variance,"error, %"
0,1,9.967150,0.469594,0.515009,0.000021,0.060660
1,1,18.121831,0.557295,0.523068,0.000012,0.045717
2,1,1.746497,0.401032,0.506885,0.000112,0.141387
3,1,4.522174,0.737967,0.509628,0.000521,0.304990
4,1,0.695879,0.726808,0.505847,0.000488,0.295136
...,...,...,...,...,...,...
95,1,0.991546,0.723571,0.506139,0.000473,0.290422
96,1,4.189751,0.664609,0.509300,0.000241,0.207445
97,1,-0.273975,0.549261,0.504888,0.000020,0.059269
98,1,0.944640,0.333194,0.506093,0.000299,0.230939


In [150]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.027262573569551137
A = 18.50623683431114


###Для функции $y_3[t]$

In [151]:
#v1, v3
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['v1(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))
Y3_linearTable

           Y3
a3  -5.638202
b31  0.000294
b33  1.920990


,optional_col,v1(9),v3(9),y3(9),regression,variance,"error, %"
0,1,11.708970,9.967150,-49.995722,13.512043,40.332362,0.326072
1,1,-1.851966,18.121831,179.443563,29.173115,225.812075,0.771543
2,1,67.740657,1.746497,2.716082,-2.263262,0.247939,0.025566
3,1,2.999677,4.522174,2.464585,3.049734,0.003424,0.003004
4,1,6.960273,0.695879,6.726555,-4.299376,1.215712,0.056611
...,...,...,...,...,...,...,...
95,1,10.323145,0.991546,-0.274449,-3.730414,0.119437,0.017744
96,1,-0.607748,4.189751,5.949951,2.410090,0.125306,0.018175
97,1,0.995927,-0.273975,0.278633,-6.164211,0.415102,0.033080
98,1,-1.384515,0.944640,-0.831356,-3.823964,0.089557,0.015365


In [152]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 883.971654379093
A = 7.154830616210955


##2. Квадратичная форма функции пересчёта